In [ ]:
import pandas as pd
df=pd.read_csv('dataset.csv')
print(df.duplicated().sum())

0


# Task
Create a new CSV file named 'drug_smiles_risk.csv' from the dataset 'dataset.csv'. The new CSV should contain three columns: 'Drug 1 SMILES', 'Drug 2 SMILES', and 'Risk Level'. The 'Drug 1 SMILES' and 'Drug 2 SMILES' columns should contain the SMILES strings corresponding to the drug names in the 'Drug1' and 'Drug2' columns of the original dataset, respectively. The 'Risk Level' column should contain the values from the 'Risk Level' column of the original dataset.

In [ ]:
!pip install pubchempy

In [ ]:
from pubchempy import get_compounds

# Get a sample of unique drug names from the dataset
unique_drugs = pd.concat([df['Drug 1'], df['Drug 2']]).unique()
sample_drugs = unique_drugs[:10]  # Take the first 10 unique drugs as a sample

smiles_dict = {}
for drug_name in sample_drugs:
    try:
        # Search PubChem for the compound by name
        compounds = get_compounds(drug_name, 'name')
        if compounds:
            # Assuming the first result is the correct one, get the SMILES string
            smiles_dict[drug_name] = compounds[0].canonical_smiles
        else:
            smiles_dict[drug_name] = None
            print(f"Could not find SMILES for drug: {drug_name}")
    except Exception as e:
        smiles_dict[drug_name] = None
        print(f"An error occurred while searching for {drug_name}: {e}")

print("\nSample SMILES results:")
for drug, smiles in smiles_dict.items():
    print(f"{drug}: {smiles}")

# Evaluate the results - check how many SMILES were found and if they look reasonable.
# This will help determine if PubChemPy is a viable option for the entire dataset.

/tmp/ipython-input-3638512424.py:14: PubChemPyDeprecationWarning: canonical_smiles is deprecated: Use connectivity_smiles instead
  smiles_dict[drug_name] = compounds[0].canonical_smiles



Sample SMILES results:
Trioxsalen: CC1=CC(=O)OC2=C1C=C3C=C(OC3=C2C)C
Aminolevulinic acid: C(CC(=O)O)C(=O)CN
Titanium dioxide: O=[Ti]=O
Tiaprofenic acid: CC(C1=CC=C(S1)C(=O)C2=CC=CC=C2)C(=O)O
Cyamemazine: CC(CN1C2=CC=CC=C2SC3=C1C=C(C=C3)C#N)CN(C)C
Temoporfin: C1CC2=NC1=C(C3=CC=C(N3)C(=C4C=CC(=N4)C(=C5C=CC(=C2C6=CC(=CC=C6)O)N5)C7=CC(=CC=C7)O)C8=CC(=CC=C8)O)C9=CC(=CC=C9)O
Methoxsalen: COC1=C2C(=CC3=C1OC=C3)C=CC(=O)O2
Hexaminolevulinate: CCCCCCOC(=O)CCC(=O)CN
Benzophenone: C1=CC=C(C=C1)C(=O)C2=CC=CC=C2
Riboflavin: CC1=CC2=C(C=C1C)N(C3=NC(=O)NC(=O)C3=N2)CC(C(C(CO)O)O)O


In [ ]:
drug_smiles_map = {}
unique_drug_names = pd.concat([df['Drug 1'], df['Drug 2']]).unique()

In [ ]:
from pubchempy import get_compounds
import time

for drug_name in unique_drug_names:
    if drug_name in drug_smiles_map:
        continue  # Skip if already processed

    try:
        compounds = get_compounds(drug_name, 'name')
        if compounds:
            drug_smiles_map[drug_name] = compounds[0].canonical_smiles
        else:
            drug_smiles_map[drug_name] = None
            print(f"Could not find SMILES for drug: {drug_name}")
    except Exception as e:
        drug_smiles_map[drug_name] = None
        print(f"An error occurred while searching for {drug_name}: {e}")
    # Add a small delay to avoid hitting API rate limits
    time.sleep(0.1)

print(f"Finished processing. Found SMILES for {sum(value is not None for value in drug_smiles_map.values())} out of {len(unique_drug_names)} unique drugs.")

/tmp/ipython-input-287457564.py:11: PubChemPyDeprecationWarning: canonical_smiles is deprecated: Use connectivity_smiles instead
  drug_smiles_map[drug_name] = compounds[0].canonical_smiles


Could not find SMILES for drug: Polythiazide
Could not find SMILES for drug: Radium Ra 223 dichloride
Could not find SMILES for drug: Verteporfin
Could not find SMILES for drug: Sucralfate
Could not find SMILES for drug: Kaolin
Could not find SMILES for drug: Nitroprusside
Could not find SMILES for drug: Pentosan polysulfate
Could not find SMILES for drug: Mipomersen
Could not find SMILES for drug: Polymyxin B
Finished processing. Found SMILES for 1692 out of 1701 unique drugs.


## Create new dataframe

### Subtask:
Construct a new pandas DataFrame with three columns: 'Drug 1 SMILES', 'Drug 2 SMILES', and 'Risk Level'.


**Reasoning**:
Create an empty dictionary with the specified column names as keys, and initialize each key with an empty list, then create a pandas DataFrame from this dictionary.



In [ ]:
new_data = {
    'Drug 1 SMILES': [],
    'Drug 2 SMILES': [],
    'Risk Level': []
}
new_df = pd.DataFrame(new_data)

## Populate dataframe

### Subtask:
Populate the new DataFrame with the retrieved SMILES strings for Drug 1 and Drug 2, and the Risk Level from the original dataset.


**Reasoning**:
Iterate through the original dataframe, retrieve the SMILES strings for each drug using the pre-populated drug_smiles_map, and the risk level, then append them to the new_df dataframe.



In [ ]:
# Filter out rows where either Drug 1 SMILES or Drug 2 SMILES is missing
new_df_filtered = new_df.dropna(subset=['Drug 1 SMILES', 'Drug 2 SMILES'])

# If 'Not Found' was used as a placeholder for missing values, filter those out as well
if 'Not Found' in new_df_filtered['Drug 1 SMILES'].unique() or 'Not Found' in new_df_filtered['Drug 2 SMILES'].unique():
    new_df_filtered = new_df_filtered[
        (new_df_filtered['Drug 1 SMILES'] != 'Not Found') &
        (new_df_filtered['Drug 2 SMILES'] != 'Not Found')
    ]

display(new_df_filtered.head())

# Save the filtered DataFrame to a CSV file
new_df_filtered.to_csv('drug_smiles_risk.csv', index=False)

print("\n✅ Updated CSV created successfully: 'drug_smiles_risk.csv'")

,Drug 1 SMILES,Drug 2 SMILES,Risk Level



✅ Updated CSV created successfully: 'drug_smiles_risk.csv'


In [ ]:
import pandas as pd

# Load the original dataset
df = pd.read_csv('dataset.csv')

# Continue with populating the new_df DataFrame and saving the filtered data
new_data_list = []
rows_to_keep = [] # Keep track of indices to keep

for index, row in df.iterrows():
    drug1_name = row['Drug 1']
    drug2_name = row['Drug 2']
    risk_level = row['Risk Level']

    # Assuming drug_smiles_map is already populated from a previous step
    drug1_smiles = drug_smiles_map.get(drug1_name, None)
    drug2_smiles = drug_smiles_map.get(drug2_name, None)

    # Only include rows where both SMILES were found
    if drug1_smiles is not None and drug2_smiles is not None:
        new_row = {
            'Drug 1 SMILES': drug1_smiles,
            'Drug 2 SMILES': drug2_smiles,
            'Risk Level': risk_level
        }
        new_data_list.append(new_row)
        rows_to_keep.append(index)


new_df = pd.DataFrame(new_data_list)

# Filter out rows where either Drug 1 SMILES or Drug 2 SMILES is missing
# This is already handled by the 'if' condition above, but keeping the filter for clarity
new_df_filtered = new_df.dropna(subset=['Drug 1 SMILES', 'Drug 2 SMILES'])

# If 'Not Found' was used as a placeholder for missing values, filter those out as well
if 'Not Found' in new_df_filtered['Drug 1 SMILES'].unique() or 'Not Found' in new_df_filtered['Drug 2 SMILES'].unique():
    new_df_filtered = new_df_filtered[
        (new_df_filtered['Drug 1 SMILES'] != 'Not Found') &
        (new_df_filtered['Drug 2 SMILES'] != 'Not Found')
    ]


display(new_df_filtered.head())

# Save the filtered DataFrame to a CSV file
new_df_filtered.to_csv('drug_smiles_risk.csv', index=False)

print("\n✅ Updated CSV created successfully: 'drug_smiles_risk.csv'")

# Drop rows from the original dataframe where SMILES were not found
df_filtered_original = df.loc[rows_to_keep].reset_index(drop=True)

# Save the filtered original dataframe to a new CSV file
df_filtered_original.to_csv('dataset_filtered.csv', index=False)

print("\n✅ Filtered original dataset created successfully: 'dataset_filtered.csv'")

,Drug 1 SMILES,Drug 2 SMILES,Risk Level
0,C(CC(=O)O)C(=O)CN,CC1C(C(CC(O1)OC2C(OC(CC2O)OC3C(OC(CC3O)OC4CCC5...,high
1,C1CC2=NC1=C(C3=CC=C(N3)C(=C4C=CC(=N4)C(=C5C=CC...,CC1C(C(CC(O1)OC2C(OC(CC2O)OC3C(OC(CC3O)OC4CCC5...,high
2,C1CNP(=O)(OC1)N(CCCl)CCCl,CC1C(C(CC(O1)OC2C(OC(CC2O)OC3C(OC(CC3O)OC4CCC5...,high
3,CC1=C2C(C(=O)C3(C(CC4C(C3C(C(C2(C)C)(CC1OC(=O)...,CC1C(C(CC(O1)OC2C(OC(CC2O)OC3C(OC(CC3O)OC4CCC5...,high
4,CC1=C2C(C(=O)C3(C(CC4C(C3C(C(C2(C)C)(CC1OC(=O)...,CC1C(C(CC(O1)OC2C(OC(CC2O)OC3C(OC(CC3O)OC4CCC5...,high



✅ Updated CSV created successfully: 'drug_smiles_risk.csv'

✅ Filtered original dataset created successfully: 'dataset_filtered.csv'


The 'drug_smiles_risk.csv' file contains 190422 rows.
